# Chap07.2 - 텐서플로 추상화와 간소화, TFLearn

> **TFLearn**은 [Chap07.1 Estimator](http://excelsior-cjh.tistory.com/157)에서 살펴본 `tf.estimator`와 마찬가지로 텐서플로의 추상화 라이브러리이다. 이번에는 TFLearn에 대해 알아보도록 하자.

## 7.3 TFLearn

### 7.3.1 설치

[**TFLearn**](http://tflearn.org/)은 텐서플로에 포함되어 있지 않기 때문에 별도의 설치가 필요하다. Terminal(또는 cmd창)에 `pip` 명령을 이용해 설치할 수 있다.

```bash
pip install tflearn
```